In [1]:
import numpy as np

In [2]:
from sklearn.datasets import fetch_openml

X, y = fetch_openml('titanic', version=1, return_X_y=True, as_frame=True)

In [3]:
X.head()

,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1.0,"Allen, Miss. Elisabeth Walton",female,29.0000,0.0,0.0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1.0,"Allison, Master. Hudson Trevor",male,0.9167,1.0,2.0,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1.0,"Allison, Miss. Helen Loraine",female,2.0000,1.0,2.0,113781,151.5500,C22 C26,S,None,NaN,"Montreal, PQ / Chesterville, ON"
3,1.0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1.0,2.0,113781,151.5500,C22 C26,S,None,135.0,"Montreal, PQ / Chesterville, ON"
4,1.0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1.0,2.0,113781,151.5500,C22 C26,S,None,NaN,"Montreal, PQ / Chesterville, ON"


In [4]:
X.drop(['name', 'home.dest', 'cabin', 'boat', 'body', 'ticket'], axis=1, inplace=True)

In [5]:
X.head()

,pclass,sex,age,sibsp,parch,fare,embarked,boat
0,1.0,female,29.0000,0.0,0.0,211.3375,S,2
1,1.0,male,0.9167,1.0,2.0,151.5500,S,11
2,1.0,female,2.0000,1.0,2.0,151.5500,S,None
3,1.0,male,30.0000,1.0,2.0,151.5500,S,None
4,1.0,female,25.0000,1.0,2.0,151.5500,S,None


In [6]:
numerical_features = list(X.dtypes[(X.dtypes != 'object') & (X.dtypes != 'category')].index)
categorical_features = list(X.dtypes[(X.dtypes == 'object') | (X.dtypes == 'category')].index)

print('Numerical features:', numerical_features)
print('Categorical features:', categorical_features)

Numerical features: ['pclass', 'age', 'sibsp', 'parch', 'fare']
Categorical features: ['sex', 'embarked', 'boat']


In [7]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

numerical_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

In [8]:
from sklearn.preprocessing import OneHotEncoder

categorical_transformer = Pipeline([
    ('onehot', OneHotEncoder())
])

In [9]:
from sklearn.compose import ColumnTransformer

column_transformer = ColumnTransformer([
    ('numerical', numerical_transformer, numerical_features),
    ('categorical', categorical_transformer, categorical_features)
])

In [10]:
from sklearn.ensemble import RandomForestClassifier

pipeline = Pipeline([
    ('column_transformer', column_transformer),
    ('clf', RandomForestClassifier())
])

In [11]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_train.head()

,pclass,sex,age,sibsp,parch,fare,embarked,boat
900,3.0,male,NaN,1.0,2.0,23.4500,S,None
658,3.0,female,0.75,2.0,1.0,19.2583,C,C
39,1.0,male,48.00,0.0,0.0,50.4958,C,None
9,1.0,male,71.00,0.0,0.0,49.5042,C,None
1125,3.0,male,24.00,0.0,0.0,8.0500,S,None


In [12]:
X_test.head()

,pclass,sex,age,sibsp,parch,fare,embarked,boat
905,3.0,male,32.0,0.0,0.0,7.8542,S,15
1181,3.0,male,24.0,0.0,0.0,9.3250,S,None
1174,3.0,female,NaN,8.0,2.0,69.5500,S,None
780,3.0,female,23.0,0.0,0.0,8.0500,S,None
453,2.0,male,21.0,0.0,0.0,73.5000,S,None


In [13]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('column_transformer',
                 ColumnTransformer(transformers=[('numerical',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['pclass', 'age', 'sibsp',
                                                   'parch', 'fare']),
                                                 ('categorical',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder())]),
                                                  ['sex', 'embarked',
                                                   'boat'])])),
                ('clf', RandomForestClassifier())])

In [14]:
print('Training set accuracy:', np.round(pipeline.score(X_train, y_train), 4))
print('Test set accuracy:' , np.round(pipeline.score(X_test, y_test), 4))

Training set accuracy: 0.9981


ValueError: Found unknown categories ['8 10'] in column 2 during transform